In [68]:
!pip install lightgbm

In [81]:
#Library Imports
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

In [82]:
train=pd.read_csv('./Data_for_Modeling/Train_for_model.csv')
test=pd.read_csv('./Data_for_Modeling/Test_for_model.csv')
submission=pd.read_csv('sample_submission.csv')

In [83]:
#train.shape 122400 X 10
#60개의 건물 X 85일 24시간 =122400
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0,0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0,0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0,0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0,0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
118075,60,2020-08-21 19,3780.000,27.0,1.9,77.0,0.0,0.1,1,1
118076,60,2020-08-21 20,3691.440,26.0,2.6,82.0,0.0,0.0,1,1
118077,60,2020-08-21 21,3532.896,25.2,2.2,83.0,0.0,0.0,1,1
118078,60,2020-08-21 22,3411.504,24.9,2.4,84.0,0.0,0.0,1,1


In [84]:
#test.shape 10080 X 9
#60개의 건물 X 7일 24시간 =10080
test

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-08-22 00,8646.264,25.2,1.9,87,0.0,0.0,0,0
1,1,2020-08-22 01,8637.192,25.0,2.0,86,0.0,0.0,0,0
2,1,2020-08-22 02,8616.456,25.0,1.4,86,0.0,0.0,0,0
3,1,2020-08-22 03,8592.480,24.7,1.8,86,0.0,0.0,0,0
4,1,2020-08-22 04,8570.448,24.4,1.7,87,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
4315,60,2020-08-24 19,4114.368,27.8,2.3,68,0.0,0.7,1,1
4316,60,2020-08-24 20,3975.696,27.3,1.2,71,0.0,0.0,1,1
4317,60,2020-08-24 21,3572.208,27.3,1.8,71,0.0,0.0,1,1
4318,60,2020-08-24 22,3299.184,27.1,1.8,74,0.0,0.0,1,1


# **데이터 전처리**

In [85]:
#건물별로 '비전기냉방설비운영'과 '태양광보유'를 판단해 test set의 결측치를 보간해줍니다
train[['num', '비전기냉방설비운영','태양광보유']]
ice={}
hot={}
count=0
for i in range(0, len(train), len(train)//60):
    count +=1
    ice[count]=train.loc[i,'비전기냉방설비운영']
    hot[count]=train.loc[i,'태양광보유']
    

In [86]:
for i in range(len(test)):
    test.loc[i, '비전기냉방설비운영']=ice[test['num'][i]]
    test.loc[i, '태양광보유']=hot[test['num'][i]]

In [87]:
#시간 변수와 요일 변수를 추가해봅니다.
def time(x):
    return int(x[-2:])
train['time']=train['date_time'].apply(lambda x: time(x))
test['time']=test['date_time'].apply(lambda x: time(x))

def weekday(x):
    return pd.to_datetime(x[:10]).weekday()
train['weekday']=train['date_time'].apply(lambda x :weekday(x))
test['weekday']=test['date_time'].apply(lambda x :weekday(x))

+ test 결측치 보간해줍니다.
+ test 데이터의 변수는 예보 데이터이며, 예보 데이터는 train 데이터의 기간에 생성된 것이기에 활용 가능합니다.

In [88]:
test.interpolate(method='values')

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,time,weekday
0,1,2020-08-22 00,8646.264,25.2,1.9,87,0.0,0.0,0,0,0,5
1,1,2020-08-22 01,8637.192,25.0,2.0,86,0.0,0.0,0,0,1,5
2,1,2020-08-22 02,8616.456,25.0,1.4,86,0.0,0.0,0,0,2,5
3,1,2020-08-22 03,8592.480,24.7,1.8,86,0.0,0.0,0,0,3,5
4,1,2020-08-22 04,8570.448,24.4,1.7,87,0.0,0.0,0,0,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4315,60,2020-08-24 19,4114.368,27.8,2.3,68,0.0,0.7,1,1,19,0
4316,60,2020-08-24 20,3975.696,27.3,1.2,71,0.0,0.0,1,1,20,0
4317,60,2020-08-24 21,3572.208,27.3,1.8,71,0.0,0.0,1,1,21,0
4318,60,2020-08-24 22,3299.184,27.1,1.8,74,0.0,0.0,1,1,22,0


# **모델링**

In [89]:
train_x=train.drop('전력사용량(kWh)', axis=1)
train_y=train[['전력사용량(kWh)']]

In [90]:
train_x.drop('date_time', axis=1, inplace=True)
test.drop('date_time', axis=1, inplace=True)

In [91]:
cross=KFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in cross.split(train_x, train_y):
    folds.append((train_idx, valid_idx))

아래 코드는 최정명님의 코드 구성 방식을 살며시 활용했습니다. 감사합니다

https://www.dacon.io/competitions/official/235713/codeshare/2476?page=1&dtype=recent


In [92]:
models={}
for fold in range(5):
    print(f'===================={fold+1}=======================')
    train_idx, valid_idx=folds[fold]
    X_train=train_x.iloc[train_idx, :]
    y_train=train_y.iloc[train_idx, :]
    X_valid=train_x.iloc[valid_idx, :]
    y_valid=train_y.iloc[valid_idx, :]
    
    model=LGBMRegressor(n_estimators=100)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], 
             early_stopping_rounds=30, verbose=100)
    models[fold]=model
    
    print(f'================================================\n\n')

====================1=======================


LightGBMError: Do not support non-ascii characters in feature name.

In [ ]:
for i in range(5):
    submission['answer'] += models[i].predict(test)/5 

In [ ]:
submission

In [ ]:
#제출
submission.to_csv('baseline_submission3.csv', index=False)